In [1]:
import numpy as np
import pandas as pd
import json
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import Input, Model
from keras.layers import Dense, Dropout, Flatten, add
import keras

In [3]:
from keras.utils import to_categorical, Sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import RMSprop,Adam

In [4]:
BASE_DIR = "../input/cassava-leaf-disease-classification/"

os.listdir(BASE_DIR)

['train_tfrecords',
 'train_images',
 'test_tfrecords',
 'sample_submission.csv',
 'label_num_to_disease_map.json',
 'train.csv',
 'test_images']

In [5]:
train_data = pd.read_csv(BASE_DIR+'train.csv')
samp_subm = pd.read_csv(BASE_DIR+'sample_submission.csv')

print(train_data)

             image_id  label
0      1000015157.jpg      0
1      1000201771.jpg      3
2       100042118.jpg      1
3      1000723321.jpg      1
4      1000812911.jpg      3
...               ...    ...
21392   999068805.jpg      3
21393   999329392.jpg      3
21394   999474432.jpg      1
21395   999616605.jpg      4
21396   999998473.jpg      4

[21397 rows x 2 columns]


In [6]:
print('number of train data:', len(train_data))
print('number of train images:', len(os.listdir(BASE_DIR+'train_images/')))
print('number of test images:', len(os.listdir(BASE_DIR+'test_images/')))

number of train data: 21397
number of train images: 21397
number of test images: 1


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_df = pd.read_csv(os.path.join(BASE_DIR, 'train.csv'))
train_df['label'] = train_df['label'].astype(str)


train_datagen = ImageDataGenerator(validation_split = 0.2,  #给出验证数据分配比例（validation_split = 0.2）
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)


validation_datagen = ImageDataGenerator(validation_split = 0.2,rescale=1./255) #注意，不能增强验证数据 #给出验证数据分配比例（validation_split = 0.2）

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=os.path.join(BASE_DIR, 'train_images'),
    subset = "training",#需指定为训练数据
    x_col='image_id',
    y_col='label',
    target_size=(300, 300),
    class_mode='sparse', #使用'sparse' 因为train_df没有使用one-hot
    batch_size=32
)
        # 因为使用了binary_crossentropy损失，所以需要用二进制标签 默认 class_mode='categorical'
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=os.path.join(BASE_DIR, 'train_images'),
    subset = "validation", #需指定为验证数据
    x_col='image_id',
    y_col='label',
    target_size=(300, 300),
    class_mode='sparse', #使用'sparse' 因为train_df没有使用one-hot
    batch_size=32
)

Found 17118 validated image filenames belonging to 5 classes.
Found 4279 validated image filenames belonging to 5 classes.


In [8]:
BATCH_SIZE=32
STEPS_PER_EPOCH = len(train_df)*0.8 // BATCH_SIZE
VALIDATION_STEPS = len(train_df)*0.2 // BATCH_SIZE
print(STEPS_PER_EPOCH)
print(VALIDATION_STEPS)

534.0
133.0


In [9]:
resnet50_weights_dir =  "../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_resnet50 = ResNet50(weights = resnet50_weights_dir, include_top = False, input_shape = (300,300,3))
base_resnet50.trainable = False


In [10]:
def create_base_model(base_model):
    
    inputs = Input(shape = (300,300,3))
    
    base_out = base_model(inputs)
    
    base_out = Flatten()(base_out)
    
    base_out = Dropout(0.3)(base_out)
    
    base1_out = Dense(1024, activation = 'relu')(base_out)
    
    model = Model(inputs = inputs, outputs = base1_out)
    
    return model

In [11]:
resnet50_base = create_base_model(base_resnet50)

In [12]:
resnet50_base.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 10, 10, 2048)      23587712  
_________________________________________________________________
flatten (Flatten)            (None, 204800)            0         
_________________________________________________________________
dropout (Dropout)            (None, 204800)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              209716224 
Total params: 233,303,936
Trainable params: 209,716,224
Non-trainable params: 23,587,712
_________________________________________________________________


In [13]:
inputs = Input(shape = (300,300,3))

base_out = resnet50_base(inputs)
    
outputs = Dense(512, activation = 'relu')(base_out)
    
outputs = Dropout(0.2)(outputs)
    
outputs = Dense(5, activation = 'softmax')(outputs)
    
model = Model(inputs = inputs, outputs = outputs)
    
model.summary()


Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
functional_1 (Functional)    (None, 1024)              233303936 
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 2565      
Total params: 233,831,301
Trainable params: 210,243,589
Non-trainable params: 23,587,712
_________________________________________________________________


In [14]:


model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit_generator(train_generator,
                              epochs=30,
                              steps_per_epoch=STEPS_PER_EPOCH,
                              verbose=1,
                              validation_data=validation_generator,
                              validation_steps=VALIDATION_STEPS) #加入callbacks函数

Epoch 1/30
534/534 [==============================] - 581s 1s/step - loss: 6.6046 - accuracy: 0.5949 - val_loss: 1.1780 - val_accuracy: 0.6165
Epoch 2/30
534/534 [==============================] - 551s 1s/step - loss: 1.2012 - accuracy: 0.6134 - val_loss: 1.1811 - val_accuracy: 0.6165
Epoch 3/30
534/534 [==============================] - 555s 1s/step - loss: 1.1974 - accuracy: 0.6147 - val_loss: 1.1786 - val_accuracy: 0.6161
Epoch 4/30
534/534 [==============================] - 556s 1s/step - loss: 1.1871 - accuracy: 0.6146 - val_loss: 1.1813 - val_accuracy: 0.6165
Epoch 5/30
534/534 [==============================] - 548s 1s/step - loss: 1.1958 - accuracy: 0.6145 - val_loss: 1.1790 - val_accuracy: 0.6165
Epoch 6/30
534/534 [==============================] - 558s 1s/step - loss: 1.1861 - accuracy: 0.6149 - val_loss: 1.1786 - val_accuracy: 0.6175
Epoch 7/30
534/534 [==============================] - 549s 1s/step - loss: 1.1882 - accuracy: 0.6144 - val_loss: 1.1799 - val_accuracy: 0.6156

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
filenames = []
labels = []
#for f in os.listdir(os.path.join(BASE_DIR, 'test1')):
for f in os.listdir(os.path.join(BASE_DIR, 'test_images')):
    filenames.append(f)
    labels.append('0')

test_df = pd.DataFrame({
    'image_id': filenames,
    'label': labels,
})

test_datagen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    dataframe=test_df,
    directory=os.path.join(BASE_DIR, 'test_images'),
    #directory=os.path.join(BASE_DIR, 'test1'),
    x_col='image_id',
    y_col='label',
    target_size=(240, 240)
    
)

y_pred = model.predict(test_datagen)
test_df['label'] = np.argmax(y_pred, axis=1)
test_df.to_csv('submission.csv', index=False)
print(test_df)
print('Done')